# Multinational Retail Data Centralisation
##### Tasks 3 - 8 are broken down in three key steps. Extraction, Cleaning and Upload. Throughout the document we will be storing codes into methods inside of classes of three python documents. This will allow us to reduce the amount of code in a single document.

***
## **TASK 3 - ORDERS TABLE (Database)**
Retrieve **user_data** from AWS RDS database
***

***
### <font color='lightblue'>**EXTRACTION // T3**</font>
First we need to create a method to connect to the AWS RDS database. RDS Credentials provided in file [db_creds.yaml](db_creds.yaml).
<br> We will use scripts from SQLAlchemy to make the task simpler.

In [12]:
from    sqlalchemy              import create_engine
import  pandas                  as pd
import  yaml

def     read_db_creds(name):
                with open(name, 'r') as stream:
                        credentials = yaml.safe_load(stream)
                return credentials
                   
def     init_db_engine (credentials):
                engine = create_engine(f"{'postgresql'}+{'psycopg2'}://{credentials['RDS_USER']}:{credentials['RDS_PASSWORD']}@{credentials['RDS_HOST']}:{credentials['RDS_PORT']}/{credentials['RDS_DATABASE']}")
                engine.connect()
                return engine

We will use the credentials provided to identify a series of tables inside the database.

In [13]:
from    sqlalchemy          import inspect

name            = "db_creds.yaml"
credentials     = read_db_creds(name)
engine          = init_db_engine(credentials)

def     list_db_tables(engine):
                engine.connect()
                inspector = inspect(engine)
                return inspector.get_table_names()

list_db_tables(engine)

['legacy_store_details', 'legacy_users', 'orders_table']

We can now transfer the above code into our DATA CONNECTOR Class. *[see here](data_utils.py)*
<br> The user_data is located inside the second table in the list as "legacy_users".
<br> We can now commence the extraction process.

In [22]:
from    sqlalchemy          import create_engine
from    sqlalchemy          import inspect
import  pandas              as pd
import  yaml

name            = "db_creds.yaml"
credentials     = read_db_creds(name)
engine          = init_db_engine(credentials)
table_names     = list_db_tables(engine)
column          = 1

def     read_rds_table(table_names, column, engine):
                database = pd.read_sql_table(table_names[column], engine)
                return database

read_rds_table(table_names, column, engine)


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


We will now transfer the above code into our DATA EXTRACTOR Class. *[see here](data_extraction.py)*

***
### <font color='yellow'>**CLEANING // T3**</font>
Now that we have the orders table as a PD DataFrame we need to clean up any mistakes and issues that will affect our querying.

In [91]:
import  pandas              as pd

#connected file imports
from    data_utils          import DataConnector
from    data_extraction     import DataExtractor


#import class variables
dc              = DataConnector()
de              = DataExtractor()
#source data
name            = "db_creds.yaml" 

#connect and create database
credentials     = dc.read_db_creds(name)
engine          = dc.init_db_engine(credentials)
table_names     = dc.list_db_tables(engine)
column          = 1 #user_data

database        = de.read_rds_table(table_names,column,engine)

database.info()
database

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [99]:
#CLEANING -----------------------------------------------------------------
database                        = database.drop(columns="index")
database["date_of_birth"]       = pd.to_datetime(database["date_of_birth"], "coerce", format = "%Y-%m-%d")
database["join_date"]           = pd.to_datetime(database["join_date"], "coerce", format = "%Y-%m-%d")

database.info()
database

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15320 non-null  object        
 1   last_name      15320 non-null  object        
 2   date_of_birth  15257 non-null  datetime64[ns]
 3   company        15320 non-null  object        
 4   email_address  15320 non-null  object        
 5   address        15320 non-null  object        
 6   country        15320 non-null  object        
 7   country_code   15320 non-null  object        
 8   phone_number   15320 non-null  object        
 9   join_date      15261 non-null  datetime64[ns]
 10  user_uuid      15320 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.3+ MB


,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...
15315,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


Data Cleaning Complete.
Now that we know what to clean, we can take the operations above and compile a method inside our [data_cleaning.py](data_cleaning.py) class called ***DataCleaning***.
We will call this method ***clean_users_data***.

***
### <font color='lightgreen'>**UPLOAD TO SCHEMA // T3**</font>
<sub><sup>This CELL will contain all operations of extraction, cleaning and uploading within a single operation in order to streamline future updates.</sup></sub>

In [1]:
from    data_utils          import  DataConnector
from    data_extraction     import  DataExtractor
from    data_cleaning       import  DataCleaning


def upload_to_dim_users():
        #CLASS VARIABLES
        dc              = DataConnector()
        de              = DataExtractor()
        dcl             = DataCleaning()
                
        #EXTRACTION
        name            = "db_creds.yaml"
        credentials     = dc.read_db_creds(name)
        engine          = dc.init_db_engine(credentials)
        table_names     = dc.list_db_tables(engine)
        column          = 1 #user_data
        
        database        = de.read_rds_table(table_names,column,engine)

        #CLEANING
        database        = dcl.clean_user_data(database)

        #SCHEMA SERVER
        sql_name        = "dim_users"
        local_name      = "db_creds_local.yaml" 
        credentials     = dc.read_db_creds(local_name)
        engine          = dc.init_db_engine(credentials)
        
        #UPLOAD
        dc.upload_to_db(database, sql_name, engine)

upload_to_dim_users()

***
## **TASK 4 - CARD DETAILS (Database)**
Retrieve **card_details** from PDF document located in AWS S3 Bucket.
<br>
PDF LINK: https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf
***

***
### <font color='lightblue'>**EXTRACTION // T4**</font>
Extracting from a S3 Address means we have to create our own BUCKET to store the database. We'll have to log into AWS CLI in order to download the file. <br> In order to create the database table, we are extracting from the amazon S3 server. <br> We will use the method from the Boto3 1.33.12 Documentation, *https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html*

***
### <font color='yellow'>**CLEANING // T4**</font>
Now that we have the orders table as a PD DataFrame we need to clean up any mistakes and issues that will affect our querying.

***
### <font color='lightgreen'>**UPLOAD TO SCHEMA // T4**</font>
<sub><sup>This CELL will contain all operations of extraction, cleaning and uploading within a single operation in order to streamline future updates.</sup></sub>

In [ ]:
from    data_utils          import  DataConnector
from    data_extraction     import  DataExtractor
from    data_cleaning       import  DataCleaning


def upload_to_dim_card_details():
        #CLASS VARIABLES
        dc              = DataConnector()
        de              = DataExtractor()
        dcl             = DataCleaning()
        #EXTRACTION
        link            = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf" 
               
        #connect and create database
        database        = de.retreive_pdf_data(link)

        #CLEANING
        database        = dcl.clean_card_details(database)

        #SCHEMA SERVER
        sql_name        = "dim_card_details"
        local_name      = "db_creds_local.yaml" 
        credentials     = dc.read_db_creds(local_name)
        engine          = dc.init_db_engine(credentials)
        
        #UPLOAD
        dc.upload_to_db(database, sql_name, engine)

upload_to_dim_card_details()

***
## **TASK 5 - STORE DATA (Database)**
Retrieve **store_data** & **store_details** via API.
<br>
Retrieve a store: https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}
<br>
Return the number of stores: https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores
***

***
### <font color='lightblue'>**EXTRACTION // T5**</font>
Extracting from a S3 Address means we have to create our own BUCKET to store the database. We'll have to log into AWS CLI in order to download the file. <br> In order to create the database table, we are extracting from the amazon S3 server. <br> We will use the method from the Boto3 1.33.12 Documentation, *https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html*

***
### <font color='yellow'>**CLEANING // T5**</font>
Now that we have the orders table as a PD DataFrame we need to clean up any mistakes and issues that will affect our querying.

***
### <font color='lightgreen'>**UPLOAD TO SCHEMA // T5**</font>
<sub><sup>This CELL will contain all operations of extraction, cleaning and uploading within a single operation in order to streamline future updates.</sup></sub>

In [ ]:
from    data_utils          import  DataConnector
from    data_extraction     import  DataExtractor
from    data_cleaning       import  DataCleaning


def     upload_to_dim_store_details():
                #CLASS VARIABLES
                dc              = DataConnector()
                de              = DataExtractor()
                dcl             = DataCleaning()
                
                #EXTRACTION
                database        = de.retrieve_store_data()

                #CLEANING
                database        = dcl.clean_store_data(database)

                #SCHEMA SERVER
                sql_name        = "dim_store_details"
                local_name      = "db_creds_local.yaml" 
                credentials     = dc.read_db_creds(local_name)
                engine          = dc.init_db_engine(credentials)
                
                #UPLOAD
                dc.upload_to_db(database, sql_name, engine)

upload_to_dim_store_details()

***
## **TASK 6 - PRODUCT INFO (Database)**
Retrieve **prdocuts** from CSV document located in AWS S3 Bucket.
<br>
S3 Address: s3://data-handling-public/products.csv
***

***
### <font color='lightblue'>**EXTRACTION // T6**</font>
Extracting from a S3 Address means we have to create our own BUCKET to store the database. We'll have to log into AWS CLI in order to download the file. <br> In order to create the database table, we are extracting from the amazon S3 server. <br> We will use the method from the Boto3 1.33.12 Documentation, *https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html*

In [ ]:
import boto3

s3 = boto3.client('s3')
s3.download_file('BUCKET_NAME', 'OBJECT_NAME', 'FILE_NAME')

The S3 Address provided, **s3://data-handling-public/products.csv**, gives us a few pieces of information that we require.

In [5]:
BUCKET_NAME     = "data-handling-public"
OBJECT_NAME     = "products.csv"

The remaining FILE NAME can be any name which will be used to save as the downloaded file. (i.e. The name of the downloaded file)

In [6]:
FILE_NAME       = "jav-products.csv"

Since the final output will be a CSV file. We will require a way to read the CSV file as a Panda DataFrame. After which we can store the 3 total steps into a method function and store it away in our **DataExtractor** class inside [data_extraction.py](data_extraction.py)

In [31]:
import  pandas              as pd
import  boto3

def     extract_from_s3(BUCKET_NAME, OBJECT_NAME, FILE_NAME ):
                s3 = boto3.client("s3")
                s3.download_file(BUCKET_NAME, OBJECT_NAME, FILE_NAME )  
                df = pd.read_csv(FILE_NAME)

                return df

database = extract_from_s3(BUCKET_NAME, OBJECT_NAME, FILE_NAME)
database

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


***
### <font color='yellow'>**CLEANING // T6**</font>
Now that we have the orders table as a PD DataFrame we need to clean up any mistakes and issues that will affect our querying.

In [20]:
from    data_utils          import DataConnector
from    data_extraction     import DataExtractor

import  pandas              as pd

dc              = DataConnector()
de              = DataExtractor()
#source data
BUCKET_NAME     = "data-handling-public"
OBJECT_NAME     = "products.csv"
FILE_NAME       = "jav-products.csv"

#connect and create database
database        = de.extract_from_s3(BUCKET_NAME, OBJECT_NAME, FILE_NAME)

# ---------------------------------------------------------------------------------------

database.info()
database

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1853 non-null   int64 
 1   product_name   1849 non-null   object
 2   product_price  1849 non-null   object
 3   weight         1849 non-null   object
 4   category       1849 non-null   object
 5   EAN            1849 non-null   object
 6   date_added     1849 non-null   object
 7   uuid           1849 non-null   object
 8   removed        1849 non-null   object
 9   product_code   1849 non-null   object
dtypes: int64(1), object(9)
memory usage: 144.9+ KB


,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


From initial inspections, it is clear we have a few pieces of Data that need to be corrected. <br>
    <ol>
    1. Remove Column "Unnamed: 0" <br>
    2. Weight Column needs to have uniform weight. <br> 
    3. Adjusting data types across the board <br>
    4. Renaming the uuid column into product_uuid as there are other uuid data across the series. <br>
    5. Remove duplicates and nulls <br>
    </ol>

In [2]:
from    data_utils          import DataConnector
from    data_extraction     import DataExtractor

import  pandas              as pd
import  numpy               as np

dc              = DataConnector()
de              = DataExtractor()
#source data
BUCKET_NAME     = "data-handling-public"
OBJECT_NAME     = "products.csv"
FILE_NAME       = "jav-products.csv"

#connect and create database
database        = de.extract_from_s3(BUCKET_NAME, OBJECT_NAME, FILE_NAME)

# ---------------------------------------------------------------------------------------
#CLEANING PROCESS

database = database.rename(columns={"uuid":"product-uuid"})
#added index_col=[0] to the data_extractor method for reading the csv.

database[["kilo-weight", "unit"]]               = (database["weight"].str.split("kg", n=1, expand=True))
#database[["Multiplier", "unit-weight"]]         = (database["weight"].str.split(" x ", n=1, expand=True))
database[["gram-weight", "unit"]]               = (database["weight"].str.split("g", n=1, expand=True))
#database[["gram-weight", "unit"]]               = (database["unit-weight"].str.split("g", n=1, expand=True))
database[["ml-weight", "unit"]]                 = (database["weight"].str.split("ml", n=1, expand=True))
database[["oz-weight", "unit"]]                 = (database["weight"].str.split("oz", n=1, expand=True))

database                                        = database.drop(columns=["unit"])

database["kilo-weight"]             = pd.to_numeric(database["kilo-weight"], "coerce", "integer", dtype_backend="numpy_nullable")
database["gram-weight"]             = pd.to_numeric(database["gram-weight"], "coerce", "integer", dtype_backend="numpy_nullable")
database["ml-weight"]               = pd.to_numeric(database["ml-weight"], "coerce", "integer", dtype_backend="numpy_nullable")
database["oz-weight"]               = pd.to_numeric(database["oz-weight"], "coerce", "integer", dtype_backend="numpy_nullable")


database.info()
database.loc[[1 , 1400 , 1401 , 1700 , 1708 , 1709 , 1800 , 1841]]


<class 'pandas.core.frame.DataFrame'>
Index: 1853 entries, 0 to 1852
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_name   1849 non-null   object 
 1   product_price  1849 non-null   object 
 2   weight         1849 non-null   object 
 3   category       1849 non-null   object 
 4   EAN            1849 non-null   object 
 5   date_added     1849 non-null   object 
 6   product-uuid   1849 non-null   object 
 7   removed        1849 non-null   object 
 8   product_code   1849 non-null   object 
 9   kilo-weight    954 non-null    Float64
 10  gram-weight    854 non-null    Float64
 11  ml-weight      9 non-null      Int16  
 12  oz-weight      1 non-null      Int8   
dtypes: Float64(2), Int16(1), Int8(1), object(9)
memory usage: 186.4+ KB


,product_name,product_price,weight,category,EAN,date_added,product-uuid,removed,product_code,kilo-weight,gram-weight,ml-weight,oz-weight
1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l,0.48,<NA>,<NA>,<NA>
1400,LB3D71C025,ODPMASE7V7,MX180RYSHX,WVPMHZP59U,BHPF2JTNKQ,PEPWA0NCVH,VIBLHHVPMN,H5N71TV8AY,OPSD21HN67,<NA>,<NA>,<NA>,<NA>
1401,Esme Stripe Woven Cushion - Mono,£14.00,0.34kg,homeware,8658228006544,2006-11-22,6d8cdd0c-ed45-4478-b34e-ac086755c4b0,Still_avaliable,N3-9339345y,0.34,<NA>,<NA>,<NA>
1700,John West Pink Salmon in Brine 3 x 132g,£3.99,3 x 132g,food-and-drink,4758183010205,2006-07-19,6f5b78f6-02cd-4ef1-ba53-678109bf9294,Still_avaliable,L5-106382J,<NA>,<NA>,<NA>,<NA>
1708,Heinz Tomato Ketchup 300ml,£1.29,300ml,food-and-drink,5540485643880,2010-07-07,bff35116-0afc-4e9b-9218-cd2ff8276684,Still_avaliable,Y0-2101400p,<NA>,<NA>,300,<NA>
1709,Heinz Seriously Good Mayonnaise 800ml,£2.79,800ml,food-and-drink,9425933566156,1994-05-01,a5e2458f-8574-4184-a4a3-382b5d504ba4,Still_avaliable,z2-6315766u,<NA>,<NA>,800,<NA>
1800,Batchelors Super Noodles 90g Chicken Flavour,£0.75,90g,food-and-drink,7773960868661,2012-05-02,af62fabe-cb03-484b-b014-a900fb836ebf,Still_avaliable,A3-4293783p,<NA>,90.0,<NA>,<NA>
1841,Stanley Fibreglass Claw Hammer 16oz,£8.99,16oz,diy,928787441710,1995-04-08,692e38f6-0949-492f-9012-61c728568272,Still_avaliable,A8-4686892S,<NA>,<NA>,<NA>,16


In [ ]:
#CONCERNS WITH ROWS 1400, 1701, 1708 and there is a random oz somwhere. 

***
### <font color='lightgreen'>**UPLOAD TO SCHEMA // T6**</font>
<sub><sup>This CELL will contain all operations of extraction, cleaning and uploading within a single operation in order to streamline future updates.</sup></sub>

In [ ]:
from    data_utils          import  DataConnector
from    data_extraction     import  DataExtractor
from    data_cleaning       import  DataCleaning


def     upload_to_dim_products():
                #CLASS VARIABLES
                dc              = DataConnector()
                de              = DataExtractor()
                dcl             = DataCleaning()

                #EXTRACTION
                BUCKET_NAME     = "data-handling-public"
                OBJECT_NAME     = "products.csv"
                FILE_NAME       = "jav-products.csv"
                
                database        = de.extract_from_s3(BUCKET_NAME, OBJECT_NAME, FILE_NAME)

                #CLEANING
                database        = dcl.clean_products_data(database)

                #SCHEMA SERVER
                sql_name        = "dim_products"
                local_name      = "db_creds_local.yaml" 
                credentials     = dc.read_db_creds(local_name)
                engine          = dc.init_db_engine(credentials)
                
                #UPLOAD
                dc.upload_to_db(database, sql_name, engine)

upload_to_dim_products()

***
## **TASK 7 - ORDERS TABLE (Database)**
Retrieve the **orders table** from an YAML Document
***

***
### <font color='lightblue'>**EXTRACTION // T7**</font>
Extracting from a YAML document has already been done in TASK 3, so we will only need to call the same method but adjust a few details such as names. <br> DATA EXTRACTOR Class. *[see here](data_extraction.py)*

In [2]:
from    data_utils          import DataConnector
from    data_extraction     import DataExtractor

import  pandas              as pd

dc              = DataConnector()
de              = DataExtractor()

name            = "db_creds.yaml" 
credentials     = dc.read_db_creds(name)
engine          = dc.init_db_engine(credentials)
table_names     = dc.list_db_tables(engine)
column          = 2

dc.list_db_tables(engine)
database        = de.read_rds_table(table_names,column, engine)
database

,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
0,0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
120118,110549,110548,f0e8fff6-9998-4661-954b-0e258e09d33c,None,None,95c74b0a-d495-4359-b1c0-e2da511e8403,575421945446,KA-FA7ED3B8,C9-6827622o,NaN,4
120119,82164,82164,1c80940a-d186-4ba9-9daa-8abd1aceae32,None,None,5d6fa6fe-e583-4baf-8bbb-d1dd6e2b551f,4971858637664481,WA-A41DA979,I0-1146408B,NaN,1
120120,97599,97599,58598aca-049c-418e-8e39-46327634a7f1,Sharon,Miller,48b7f1fc-db13-4611-ad8e-3dac0b759488,4971858637664481,WEB-1388012W,A4-5443400b,NaN,4
120121,106591,106591,3a76f661-0707-4fbc-9862-f21d3249f581,None,None,51c0b538-7ded-4697-8e84-9f7aa13f9112,4971858637664481,SO-6D328417,E9-2782979e,NaN,4


***
### <font color='yellow'>**CLEANING // T7**</font>
Now that we have the orders table as a PD DataFrame we need to clean up any mistakes and issues that will affect our querying.

In [3]:
from    data_utils          import DataConnector
from    data_extraction     import DataExtractor

import  pandas              as pd

dc              = DataConnector()
de              = DataExtractor()

'''
Below are a list of quick expressions to help investigate the dataframe

database.describe()
database.count()
database.isna().sum()
database.isnull().sum()
database.info()
database.dtypes
database.sample(20)
database.loc[database["user_uuid"] == "745a4046-3437-4a58-a307-9c88485f45f8"]
print(database["user_uuid"].value_counts())
'''

database        = de.read_rds_table(table_names,column, engine)
database        = database.drop(columns= ["level_0", "index", "1"] )

database        = database.dropna()

database.info()
database


<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 5 to 120120
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date_uuid         15284 non-null  object
 1   first_name        15284 non-null  object
 2   last_name         15284 non-null  object
 3   user_uuid         15284 non-null  object
 4   card_number       15284 non-null  int64 
 5   store_code        15284 non-null  object
 6   product_code      15284 non-null  object
 7   product_quantity  15284 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 1.0+ MB


,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,product_quantity
5,c3a1df8f-6918-4795-9f8a-6869b2cda9cf,Brett,Welch,53d21f46-1fa4-452f-a023-26aee2aae4d6,3506661913512980,WEB-1388012W,g3-7974400s,2
8,38ebd7b6-b1d4-462e-bfd7-265e3674ef3f,Marion,Stokes,02de2416-4baf-42ad-bae6-d716eca0fc3f,6011037917693140,WEB-1388012W,i4-2651057I,5
11,2e78a288-31c6-4a2a-8aea-cb93ad389aaa,Kreszentia,Hornich,bf86d13b-882b-485e-addd-0c4e9a39a96c,6502495513721380,WEB-1388012W,S6-3678717g,1
13,07248e1a-da56-4e94-8510-a9a47f80e0e2,Ramona,Eimer,196c8554-5df5-4519-973c-e05c0781cf52,371493449732930,WEB-1388012W,t5-7763922W,6
25,33c0fd7f-1d17-4b2a-9589-2e9d0c3ac8a3,Mandy,Murray,10730690-b150-4328-83c9-023d594e9215,370629659061970,WEB-1388012W,w4-86578A,4
...,...,...,...,...,...,...,...,...
120091,f5362b67-0eac-41b7-8d11-439e1c25656f,Gerhard,Jähn,2bcaa051-54ae-482d-8438-4d347b780806,3556268655280464,WEB-1388012W,Y4-6952691s,1
120104,16ec6ee4-da03-4330-8e59-e927dfc37c57,Amanda,Wheeler,85e7667e-aa97-4520-bc09-cb18b76e782c,4222069242355461965,WEB-1388012W,a3-3497174d,4
120106,826fcd34-2886-493b-9f05-071c0c73bb8c,Lee,Thomas,a2ddc7d8-3434-4a3d-920f-17befdee38a3,4971858637664481,WEB-1388012W,V2-1615331d,4
120110,1f049481-3fa2-498b-8a60-66b60a71d376,Friedemann,Jüttner,2e5d948f-3a52-42c5-ab33-3a162208915b,4971858637664481,WEB-1388012W,o6-5976806E,2


Data Cleaning Complete.
Now that we know what to clean, we can take the operations above and compile a method inside our [data_cleaning.py](data_cleaning.py) class called ***DataCleaning***.
We will call this method ***clean_orders_data***.

***
### <font color='lightgreen'>**UPLOAD TO SCHEMA // T7**</font>
<sub><sup>This CELL will contain all operations of extraction, cleaning and uploading within a single operation in order to streamline future updates.</sup></sub>

In [71]:
from    data_utils         import DataConnector
from    data_extraction    import DataExtractor
from    data_cleaning      import DataCleaning


def     upload_to_dim_orders_table():
                #CLASS VARIABLES
                dc              = DataConnector()
                de              = DataExtractor()
                dcl             = DataCleaning()

                #EXTRACTION
                name            = "db_creds.yaml" 
                credentials     = dc.read_db_creds(name)
                engine          = dc.init_db_engine(credentials)
                table_names     = dc.list_db_tables(engine)
                column          = 2

                database        = de.read_rds_table(table_names,column, engine)
                
                #CLEANING
                database        = dcl.clean_orders_data(database)
                
                #SCHEMA SERVER
                sql_name        = "orders_table"
                local_name      = "db_creds_local.yaml" 
                credentials     = dc.read_db_creds(local_name)
                engine          = dc.init_db_engine(credentials)

                #UPLOAD
                dc.upload_to_db(database, sql_name, engine)

upload_to_dim_orders_table()

***
## **TASK 8 - EVENT DATES (Database)**
Obtain Event/Activies Dates & Time for purchases orders from an AWS RDS via URL
***

### <font color='lightblue'>**EXTRACTION // T8**</font>
We have created a function using the api method to manage this connection. <br> In order to create this table, we are extracting from the amazon S3 server as URL. <br> We will use the method from the Boto3 1.33.12 Documentation, *https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-presigned-urls.html*

In [7]:
import  pandas              as pd
import  requests

LINK    = "https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json"
def     extract_from_s3_LINK():
                if LINK is not None:
                    response    = requests.get(LINK)
                    data        = pd.DataFrame(response.json())
                return data

extract_from_s3_LINK()

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb


We can now transfer the above code into our DATA EXTRACTOR Class. *[see here](data_extraction.py)*

***
### <font color='yellow'>**CLEANING // T8**</font>
Now that we have the event dates table as a PD DataFrame we need to clean up any mistakes and issues that will affect our querying.


In [77]:
from    data_extraction     import  DataExtractor


#CLASS VARIABLES
de              = DataExtractor()

#EXTRACTION
LINK            = "https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json"
database        = de.extract_from_s3_LINK(LINK)

#CLEANING
'''
Below are a list of quick expressions to help investigate the dataframe
database.describe()
database.count()
database.isna().sum()
database.isnull().sum()
database.info()
database.dtypes

This is an alternative way in removing the decimals from a column value. Ofcourse, using dtype_backend appears to be more smoother and efficient
#database["month"]       = database["month"].astype(np.int64)
'''

database["month"]       = pd.to_numeric(database["month"], "coerce", "integer", dtype_backend="numpy_nullable")
database["year"]        = pd.to_numeric(database["year"], "coerce", "integer", dtype_backend="numpy_nullable")
database["day"]         = pd.to_numeric(database["day"], "coerce", "integer", dtype_backend="numpy_nullable")
database["timestamp"]   = pd.to_datetime(database["timestamp"], "coerce", format = "%H:%M:%S").dt.time
database                = database.dropna()


database.info()
database

<class 'pandas.core.frame.DataFrame'>
Index: 120123 entries, 0 to 120160
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   timestamp    120123 non-null  object
 1   month        120123 non-null  Int8  
 2   year         120123 non-null  Int16 
 3   day          120123 non-null  Int8  
 4   time_period  120123 non-null  object
 5   date_uuid    120123 non-null  object
dtypes: Int16(1), Int8(2), object(3)
memory usage: 4.5+ MB


,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb


Data Cleaning Complete.
Now that we know what to clean, we can take the operations above and compile a method inside our [data_cleaning.py](data_cleaning.py) class called ***DataCleaning***.
We will call this method ***clean_event_dates***.

***
### <font color='lightgreen'>**UPLOAD TO SCHEMA // T8**</font>
This CELL will contain all operations of extraction, cleaning and uploading within a single operation in order to streamline future updates.

In [5]:
from    data_utils          import  DataConnector
from    data_extraction     import  DataExtractor
from    data_cleaning       import  DataCleaning


def     upload_to_dim_dates():
                #CLASS VARIABLES
                dc              = DataConnector()
                de              = DataExtractor()
                dcl             = DataCleaning()

                #EXTRACTION
                LINK            = "https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json"
                database        = de.extract_from_s3_LINK(LINK)

                #CLEANING
                database        = dcl.clean_event_dates(database)

                #SCHEMA SERVER
                sql_name        = "event_dates"
                local_name      = "db_creds_local.yaml" 
                credentials     = dc.read_db_creds(local_name)
                engine          = dc.init_db_engine(credentials)

                #UPLOAD
                dc.upload_to_db(database, sql_name, engine)

upload_to_dim_dates()